# Homework 7
## Alex Pine, 2015-12-12

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from pystruct.models import ChainCRF
from pystruct.learners import OneSlackSSVM

In [3]:
################################################################################

def read_input(data_dir, dataset_type):
    assert dataset_type == 'train' or dataset_type == 'test', dataset_type
    num_files = 5000 if dataset_type == 'train' else 1000
    
    X, y = [], []
    # Iterate over all the training sample files
    for f in [data_dir + "/Data/"+ dataset_type +"-%d.txt" % i 
              for i in range(1, num_files+1)]:
        # Read each training sample file into 'data' variable
        data = pd.read_csv(f, header=None, quoting=3)
        # Extract 'tag' field into 'labels'
        labels = data[1]
        # Extract feature fields into 'features'
        features = data.values[:, 2:].astype(np.int)
        # Adjust features starting at 1 to start at 0
        for f_idx in range(len(features)):
          f1 = features[f_idx]
          features[f_idx] = [f1[0]-1, f1[1], f1[2], f1[3]-1, f1[4]-1]
        # Adjust labels to lie in {0,...,9}, and add to 'y'
        y.append(labels.values - 1)
        # Add feature vector to 'X'
        X.append(features)

    # See: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    # [Note: if you get an error on the below line, it may be because you need to
    # upgrade scikit-learn]
    encoder = OneHotEncoder(n_values=[1,2,2,201,201],sparse=False).fit(np.vstack(X))                 
    # Represent features using one-of-K scheme: If a feature can take value in 
    # {0,...,K}, then introduce K binary features such that the value of only the 
    # i^th binary feature is non-zero when the feature takes value 'i'.
    # n_values specifies the number of states each feature can take.
    X_encoded = [encoder.transform(x) for x in X]
    return X, X_encoded, y

In [4]:
X_train_orig, X_train_enc, y_train = read_input('ps7_data', 'train')
X_test_orig, X_test_enc, y_test = read_input('ps7_data', 'test')

In [8]:
# TODO delete
# TODO what effect does the 'directed' param have? 
def learn_pos_weights(X_train, y_train, X_test, y_test, C):
    # Construct a directed ChainCRF with 10 states for each variable, 
    # and pass this CRF to OneSlackSSVM constructor to create an object 'ssvm'
    # Learn Structured SVM using X_small and y_small
    crf = ChainCRF(n_states=10, inference_method='max-product', directed=True)
    ssvm = OneSlackSSVM(crf, max_iter=200, C=C)
    ssvm.fit(X_train, y_train)
    # Store learnt weights in 'weights'
    w = ssvm.w                  
    # Evaluate training accuracy on X_small, y_small
    train_score = ssvm.score(X_train, y_train)
    # Get predicted labels on X_small using the learnt model
    # print ssvm.predict(X_small)
    test_score = ssvm.score(X_test, y_test)
    return w, train_score, test_score

# TODO consider using grid search from sklearn since it can be parallelized
def find_best_reg_param(X_train, y_train, X_test, y_test):
    params = [0.1, 1.0, 5.0] # TODO no idea what good values are, TODO expand this
    weights = []
    train_scores = []
    test_scores = []
    for C in params:
        w, train_score, test_score = learn_pos_weights(X_train, y_train, 
                                                       X_test, y_test, C)
        weights.append(w)
        train_scores.append(train_score)
        test_scores.append(test_score)
    # TODO you should probably graph this
    best_index = test_scores.index(max(test_scores))
    return (params[best_index], weights[best_index], train_scores[best_index],
            test_scores[best_index])


def find_best_model_OLD(X_train_enc, y_train, X_test_enc, y_test, num_train):
    X, y, X_val, y_val = split_train_test(X_train_enc, y_train, num_train)

    C, w, train_score, test_score = find_best_reg_param(X, y, X_val, y_val)

    full_w, full_train_score, full_test_score = learn_pos_weights(
        X_train_enc, y_train, X_test_enc, y_test, C)    
    return C, w, train_score, test_score

In [29]:
################################################################################

# Problem 1: Find the value of the regularization hyperparameter "C" that
# minimizes the loss function of the SSVM.
# To do this, we maximize the 'score' value of the SSVM.

# TODO Write a function that trains an SSVM and returns the weight vector, the 
#      test score, and the training score. Inputs should be the training and 
#      test sets.
# 
# TODO Write function that finds an optimal value of C as measured by the
#      SSVM's score function. Use grid search?
#
# TODO Find the optimal C for a model trained on the first 4500 training inputs.
#      Then report the training and test error.
#
# TODO Train a model on all the training data using the value of C you found
#      before, and report it's test error on the data in the test_XXX.txt files.
#
# TODO Make a graph of train and test error for different reg params

def split_train_test(X, y, num_train):
    TEST_SET_SIZE = 500
    assert len(X) >= num_train + TEST_SET_SIZE, len(X)
    assert len(y) >= num_train + TEST_SET_SIZE, len(y)
    return X[:num_train], y[:num_train], X[-TEST_SET_SIZE:], y[-TEST_SET_SIZE:]


def train_model(X_train, y_train, X_val, y_val, Cs):
    assert len(X_train) == len(y_train), 'x: %s y: %s' % (len(X_train), len(y_train))
    assert len(X_val) == len(y_val), 'x: %s y: %s' % (len(X_val), len(y_val))

    crf = ChainCRF(n_states=10, inference_method='max-product', directed=True)
    best_model = None
    best_C = None
    smallest_error = None
    print 'training model on data of size', len(X_train)
    for C in Cs:
        print 'training model with C =', C
        ssvm = OneSlackSSVM(crf, max_iter=200, C=C)
        ssvm.fit(X_train, y_train)
        error = 1 - ssvm.score(X_val, y_val) # TODO 1- needed?
        if not smallest_error or error < smallest_error:
            best_model = ssvm
            best_C = C
            smallest_error = error
    return best_model, best_C, smallest_error


def train_models(X, y, num_train_points_list):
    model_tuples = []
    Cs = [0.1, 1.0, 10.0] # TODOTODOTODOTODOTODOTODO
    for num_train in num_train_points_list:
        X_train, y_train, X_val, y_val = split_train_test(X, y, num_train)
        model, C, train_error = train_model(X_train, y_train, X_val, y_val, Cs)
        model_tuples.append((model, C, train_error))
    return model_tuples
    
 
def problem_one_find_best_C(X_train, y_train, X_test, y_test):
    model, C, train_error = train_models(X_train, y_train, [4500])[0]
    test_error = 1 - model.score(X_test, y_test)
    return model, C, train_error, test_error


def problem_one_full_model(X_train, y_train, X_test, y_test, best_C):
    model, C, error = train_model(X_train, y_train, X_test, y_test, [best_C])
    return model, error
    

In [30]:
print 'Training first Problem 1 model:'

prob_one_model, best_C, train_error, test_error = problem_one_find_best_C(
    X_train_enc, y_train, X_test_enc, y_test)

Training first Problem 1 model:
training model on data of size 4500
training model with C = 0.1
training model with C = 1.0
training model with C = 10.0


In [31]:
################################################################################

print 'Best C', best_C, ', training error:', train_error, ', validation error:', test_error

Best C 0.1 , training error: 0.116533949824 , validation error: 0.120678322598


In [ ]:
prob_one_full_model, test_error = problem_one_full_model(
    X_train, y_train, X_test, y_test, best_C)

In [ ]:
################################################################################

# Question 2

%matplotlib inline

import matplotlib.pyplot as plt


def train_problem_two_models(X_train, y_train, X_test, y_test, 
                             prob_one_model_tuple):
    num_train_points_list = [100, 200, 500, 1000]
    model_tuples = train_models(X_train, y_train, num_train_points_list)
    
    num_train_points_list.append(4500) # TODO arg?
    model_error_pairs.append(prob_one_tuple)
    
    models = [tup[0] for tup in model_tuples]
    Cs = [tup[1] for tup in model_tuples]
    train_errors = [tup[2] for tup in model_tuples]
    test_errors = [1 - model.score(X_test, y_test) for model in models]
    return zip(models, Cs, num_train_points_list, train_errors, test_errors)
 

In [ ]:
problem_two_model_tuple = train_problem_two_models(
    X_train, y_train, X_test, y_test, prob_one_model_tuple)
models, Cs, num_train_points_list, train_errors, test_errors = *problem_two_model_tuple


In [ ]:
def plot_errors(num_train_points_list, train_errors, test_errors):
    fig = plt.figure(1, figsize=(10, 8))
    fig.subtitle('Number of training points vs Errors', fontsize=20)
    plt.xlabel('Number of training points', fontsize=15)
    plt.ylabel('Error', fontsize=15) 
    train_line, = plt.plot(num_train_points_list, train_errors)    
    test_line, = plt.plot(num_train_points_list, test_errors)
    plt.legend([train_line, test_line], 
               ['Train Error', 'Test Error],
               loc='upper right')
    #TODOplt.yscale('log')
    plt.show()

plot_errors(num_train_points_list, train_errors, test_errors)                